<a href="https://colab.research.google.com/github/saishdesai23/Last_Mile_Delivery_Demand_Forecasting_Route_Optmization/blob/main/Last_Mile__Delivery_Route_Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installation

In [58]:
!pip install ortools

## 2. Google Drive Connection

In [59]:
# connecting google drive to the notebook
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


## 3. Import

In [60]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

## 4. Input vairables

In [61]:
# Import Distance Matrix
df_distance = pd.read_excel('/content/gdrive/MyDrive/Time Series Analysis/Store-Item demand forecasting and Route Optimization for Last Mile Delivery/data/df_distance_matrix.xlsx', index_col=0)

# Transform to Numpy Array
distance_matrix = df_distance.to_numpy()

# Create dictionnary with data
data = {}
data['distance_matrix'] = distance_matrix
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))

# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 15, 15, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 5
# Location of the depot
data['depot'] = 0

16 destinations


## 5. Optmization Algorithm

In [62]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [63]:
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

In [64]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint.
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [67]:
if solution:
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for driver {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))
else:
    print('No Solution')

Route for driver 0:
 0 Parcels(0) ->  14 Parcels(4) ->  16 Parcels(12) ->  10 Parcels(14) ->  2 Parcels(15) ->  0 Parcels(15)
Distance of the route: 2192 (m)
Parcels Delivered: 15 (parcels)

Route for driver 1:
 0 Parcels(0) ->  1 Parcels(1) ->  4 Parcels(5) ->  3 Parcels(7) ->  7 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 2:
 0 Parcels(0) ->  0 Parcels(0)
Distance of the route: 0 (m)
Parcels Delivered: 0 (parcels)

Route for driver 3:
 0 Parcels(0) ->  12 Parcels(2) ->  11 Parcels(3) ->  15 Parcels(11) ->  13 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 4:
 0 Parcels(0) ->  9 Parcels(1) ->  8 Parcels(9) ->  6 Parcels(13) ->  5 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1164 (m)
Parcels Delivered: 15 (parcels)

Total distance of all routes: 6,460 (m)
Parcels Delivered: 60/60


Reference Links

1) OR-tools
https://developers.google.com/optimization/introduction/
python

2) OR-tools example
https://colab.research.google.com/github/google/or-tools/blob/stable/examples/notebook/constraint_solver/vrp_capacity.ipynb#scrollTo=code


3) OR-tools vehile routing
https://developers.google.com/optimization/routing

4) https://towardsdatascience.com/optimize-e-commerce-last-mile-delivery-with-python-ab9ba37d214c


